In [1]:
import pandas as pd
#pd.show_versions()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1541449485506_0021,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [8]:
#ensure you have all the required libraries
%%sh
/usr/bin/anaconda/bin/pip list | grep 'azure-datalake-store\|azure-mgmt-datalake-store\|azure-mgmt-resource\|pandas\|pyarrow'

azure-datalake-store (0.0.39)
azure-mgmt-datalake-store (0.5.0)
azure-mgmt-resource (2.0.0)
pandas (0.23.4)
pyarrow (0.11.1)


You are using pip version 8.1.2, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
# import required ADL libraries
from azure.common.credentials import ServicePrincipalCredentials
from azure.datalake.store import core, lib, multithread

In [4]:
# setup ADL authentication, get param values from Azure AD
token = lib.auth(tenant_id = '72f988bf-86f1-41af-91ab-2d7cd011db47', client_secret = 'YO00AS0dOPF2u9uQutzUYCoS+loYvr5fD6jEkGANOg4=', client_id = '127fd200-1733-4782-8fa4-94c0c16a3e04')

In [5]:
# initialize ADL file system client
adlsFileSystemClient = core.AzureDLFileSystem(token, store_name='aravishdatalake')

# Read a file into pandas dataframe
# samplefile = adlsFileSystemClient.open('/aravishfolder/userdata.parquet', 'rb') 

In [6]:
#read using python ADL libraries, directly from ADLS as pandas data frame
with adlsFileSystemClient.open('/aravishfolder/userdata.parquet', 'rb') as f:
    dfpd = pd.read_parquet(f, engine='pyarrow') 

# Show the dataframe
dfpd.head()

    registration_dttm  id   ...                      title comments
0 2016-02-03 07:55:29   1   ...           Internal Auditor    1E+02
1 2016-02-03 17:04:03   2   ...              Accountant IV         
2 2016-02-03 01:09:31   3   ...        Structural Engineer         
3 2016-02-03 00:36:21   4   ...     Senior Cost Accountant         
4 2016-02-03 05:05:31   5   ...                                    

[5 rows x 13 columns]

In [17]:
#adl file object
samplefile

<ADL file: /aravishfolder/userdata.parquet>

In [10]:
#read parquet using spark sql context
df = sqlContext.read.parquet('adl://aravishdatalake.azuredatalakestore.net/aravishfolder/userdata.parquet')

In [11]:
df.printSchema()

root
 |-- registration_dttm: timestamp (nullable = true)
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- cc: string (nullable = true)
 |-- country: string (nullable = true)
 |-- birthdate: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- title: string (nullable = true)
 |-- comments: string (nullable = true)

In [12]:
df.rdd.getNumPartitions()

1

In [33]:
# Show the spark dataframe
df.show(5, True)

+-------------------+---+----------+---------+--------------------+------+--------------+----------------+------------+---------+---------+--------------------+--------+
|  registration_dttm| id|first_name|last_name|               email|gender|    ip_address|              cc|     country|birthdate|   salary|               title|comments|
+-------------------+---+----------+---------+--------------------+------+--------------+----------------+------------+---------+---------+--------------------+--------+
|2016-02-03 07:55:29|  1|    Amanda|   Jordan|    ajordan0@com.com|Female|   1.197.201.2|6759521864920116|   Indonesia| 3/8/1971| 49756.53|    Internal Auditor|   1E+02|
|2016-02-03 17:04:03|  2|    Albert|  Freeman|     afreeman1@is.gd|  Male|218.111.175.34|                |      Canada|1/16/1968|150280.17|       Accountant IV|        |
|2016-02-03 01:09:31|  3|    Evelyn|   Morgan|emorgan2@altervis...|Female|  7.161.136.94|6767119071901597|      Russia| 2/1/1960|144972.51| Structural

In [35]:
#convert spark data frame to pandas data frame
pddatafram = df.toPandas()

In [36]:
#check pandas data frame
pddatafram.head()

    registration_dttm  id   ...                      title comments
0 2016-02-03 07:55:29   1   ...           Internal Auditor    1E+02
1 2016-02-03 17:04:03   2   ...              Accountant IV         
2 2016-02-03 01:09:31   3   ...        Structural Engineer         
3 2016-02-03 00:36:21   4   ...     Senior Cost Accountant         
4 2016-02-03 05:05:31   5   ...                                    

[5 rows x 13 columns]

In [16]:
sc._conf.getAll()

[(u'spark.ui.enhancement.enabled', u'true'), (u'spark.history.kerberos.keytab', u'none'), (u'spark.eventLog.enabled', u'true'), (u'spark.yarn.containerLauncherMaxThreads', u'25'), (u'spark.driver.extraLibraryPath', u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'), (u'spark.executor.extraJavaOptions', u'-Dhdp.version= -Detwlogger.component=sparkexecutor -DlogFilter.filename=SparkLogFilters.xml -DpatternGroup.filename=SparkPatternGroups.xml -Dlog4jspark.root.logger=INFO,console,RFA,ETW,Anonymizer -Dlog4jspark.log.dir=/var/log/sparkapp/${user.name} -Dlog4jspark.log.file=sparkexecutor.log -Dlog4j.configuration=file:/usr/hdp/current/spark2-client/conf/log4j.properties -Djavax.xml.parsers.SAXParserFactory=com.sun.org.apache.xerces.internal.jaxp.SAXParserFactoryImpl -XX:+UseParallelGC -XX:+UseParallelOldGC'), (u'spark.sql.cbo.joinreorder.enabled', u'true'), (u'spark.app.id', u'application_1541449485506_0021'), (u'spark.repl.class.outputDir

In [14]:
%%configure
{"executorMemory": "3072M", "executorCores": 4, "numExecutors":10}

'SparkContext' object has no attribute 'get'
Traceback (most recent call last):
AttributeError: 'SparkContext' object has no attribute 'get'

